In [2]:
import numpy as np
import keras
import keras.backend as k
from keras.layers import Conv2D,MaxPooling2D,SpatialDropout2D,Flatten,Dropout,Dense
from keras.models import Sequential,load_model
#from keras.optimizers import Adam
from keras.preprocessing import image
import cv2
import datetime

In [32]:

model=Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(MaxPooling2D() )
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D() )
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D() )
model.add(Flatten())
model.add(Dense(100,activation='relu'))
model.add(Dense(1,activation='sigmoid'))


In [33]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])


In [34]:
from keras.preprocessing.image import ImageDataGenerator


In [35]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [36]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [37]:
training_set = train_datagen.flow_from_directory(
        'dataset/train',
        target_size=(150,150),
        batch_size=16 ,
        class_mode='binary')


Found 1315 images belonging to 2 classes.


In [38]:
test_set = test_datagen.flow_from_directory(
        'dataset/test',
        target_size=(150,150),
        batch_size=16,
        class_mode='binary')

Found 194 images belonging to 2 classes.


In [39]:
model_saved=model.fit_generator(
        training_set,
        epochs=10,
        validation_data=test_set,

        )

C:\Users\praveen solanki\AppData\Local\Temp\ipykernel_9128\2858462211.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_saved=model.fit_generator(


Epoch 1/10
83/83 [==============================] - 73s 865ms/step - loss: 0.4081 - accuracy: 0.8312 - val_loss: 0.1390 - val_accuracy: 0.9536
Epoch 2/10
83/83 [==============================] - 43s 511ms/step - loss: 0.1338 - accuracy: 0.9490 - val_loss: 0.1057 - val_accuracy: 0.9485
Epoch 3/10
83/83 [==============================] - 41s 498ms/step - loss: 0.1298 - accuracy: 0.9597 - val_loss: 0.1391 - val_accuracy: 0.9433
Epoch 4/10
83/83 [==============================] - 44s 532ms/step - loss: 0.1026 - accuracy: 0.9627 - val_loss: 0.1016 - val_accuracy: 0.9691
Epoch 5/10
83/83 [==============================] - 43s 518ms/step - loss: 0.0965 - accuracy: 0.9612 - val_loss: 0.0957 - val_accuracy: 0.9588
Epoch 6/10
83/83 [==============================] - 42s 502ms/step - loss: 0.0792 - accuracy: 0.9703 - val_loss: 0.0853 - val_accuracy: 0.9742
Epoch 7/10
83/83 [==============================] - 41s 496ms/step - loss: 0.0832 - accuracy: 0.9665 - val_loss: 0.0927 - val_accuracy: 0.9639

In [40]:

model.save('models/mask/mymodel.h5',model_saved)

In [41]:
mymodel=load_model('models/mask/mymodel.h5')
#test_image=image.load_img('C:/Users/Karan/Desktop/ML Datasets/Face Mask Detection/Dataset/test/without_mask/30.jpg',target_size=(150,150,3))
test_image=image.load_img('New_folder/test/with_mask/1-with-mask.jpg',
                          target_size=(150,150,3))

In [42]:
test_image
test_image=image.img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)
mymodel.predict(test_image)[0][0]


0.0

In [4]:
mymodel=load_model('models/mask/mymodel.h5')

In [3]:
cap=cv2.VideoCapture(0)
face_cascade=cv2.CascadeClassifier('models/cascade file/haarcascade_frontalface_default.xml')


In [5]:
while cap.isOpened():
    _,img=cap.read()
    face=face_cascade.detectMultiScale(img,scaleFactor=1.1,minNeighbors=4)
    for(x,y,w,h) in face:
        face_img = img[y:y+h, x:x+w]
        cv2.imwrite('temp.jpg',face_img)
        test_image=image.load_img('temp.jpg',target_size=(150,150,3))
        test_image=image.img_to_array(test_image)
        test_image=np.expand_dims(test_image,axis=0)
        pred=mymodel.predict(test_image)[0][0]
        if pred==1:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),3)
            cv2.putText(img,'NO MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),3)
        else:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),3)
            cv2.putText(img,'MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),3)
        datet=str(datetime.datetime.now())
        cv2.putText(img,datet,(400,450),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1)
          
    cv2.imshow('img',img)
    
    if cv2.waitKey(1)== 27:
        break
    
cap.release()
cv2.destroyAllWindows()